### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [1]:
import pandas as pd
from sklearn.metrics import r2_score
from urllib import request, parse

df = pd.read_excel("final.xlsx")
df.head()

,Date,USD course,Commitments Long-Term,Commitments Short-Term,Earnings per share,EBITDA,CB rate,FRS rate,"Nickel close price, USD","Copper close price, USD","Palladium close price, USD","Platinum close price, USD",MMVB close,"GMKN close, RUB"
0,2021-07-19,74.165604,9319.766602,3798.506348,42.369442,5901.756836,0.055,0.0025,18380,9220.865234,2597.0,1068.099976,3693.419922,25046
1,2021-07-20,74.346298,9319.766602,3798.506348,42.369442,5901.756836,0.055,0.0025,18665,9220.865234,2640.5,1067.199951,3704.399902,25138
2,2021-07-21,74.490997,9319.766602,3798.506348,42.369442,5901.756836,0.055,0.0025,18605,9220.865234,2660.0,1079.099976,3722.959961,25050
3,2021-07-22,74.494698,9319.766602,3798.506348,42.369442,5901.756836,0.055,0.0025,18945,9220.865234,2723.0,1093.500000,3754.439941,25080
4,2021-07-23,73.694504,9319.766602,3798.506348,42.369442,5901.756836,0.055,0.0025,19470,9220.865234,2672.5,1058.699951,3734.540039,24990


In [2]:
X = df.drop('GMKN close, RUB', axis = 1).copy()
y = df[['GMKN close, RUB']].copy()

In [10]:
import urllib.request
import json      

def get_prediction(dat, usd, com_long, com_short, earn, ebitds, cb_rate, frs_rate, nickel, copper, palladium, platinum, mmvb):
    body = {'Date': dat,
            'USD course': usd,
            'Commitments Long-Term': com_long,
            'Commitments Short-Term': com_short,
            'Earnings per share': earn,
            'EBITDA': ebitda,
            'CB rate': cb_rate,
            'FRS rate': frs_rate,
            'Nickel close price, USD': nickel,
            'Copper close price, USD': copper,
            'Palladium close price, USD': palladium,
            'Platinum close price, USD': platinum,
            'MMVB close': mmvb} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [7]:
%%time
predictions = X_test[['Date', 'USD course', 'Commitments Long-Term', 'Commitments Short-Term', 'Earnings per share',
                     'EBITDA', 'CB rate', 'FRS rate', 'Nickel close price, USD', 'Copper close price, USD',
                     'Palladium close price, USD', 'Platinum close price, USD', 'MMVB close']].iloc[:9].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12]), 1)

CPU times: user 340 ms, sys: 55.7 ms, total: 396 ms
Wall time: 5.16 s


In [8]:
r2_score(y_score=predictions.values, y_true=y_test.iloc[:9])

0.9803161348046525

In [7]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [9]:
get_prediction("", "", None)

0.13518841848029955